In [ ]:
#SVM

In [1]:
#Importuri
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import csv

In [2]:
#Functie de normalizare realizata in cadrul laboratorului

def normalize_data(train_data, test_data, type=None):
    scaler = None
    if type == 'standard':
        scaler = preprocessing.StandardScaler()

    elif type == 'l1':
        scaler = preprocessing.Normalizer(norm='l1')

    elif type == 'l2':
        scaler = preprocessing.Normalizer(norm='l2')

    if scaler is not None:
        scaler.fit(train_data)
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data) 
        return (scaled_train_data, scaled_test_data)
    else:
        print("No scaling was performed. Raw data is returned.")
        return (train_data, test_data)

In [3]:
#Functie care calculeaza acuratetea unui model pe baza datelor de validare

def accuracy_score(y_true, y_pred):
    return (y_true == y_pred).mean()

In [4]:
#Functie care extrage proprozitia corespunzatoare fiecarui id din setul de date

def get_sentence(data):
    new_data = []
    
    for item in data:
        new_data.append(item[1])
    
    #Vectorul cu toate propozitiile este convertit intr-un numpy array
    return np.array(new_data)

In [9]:
#Functie care extrage etichetele corespunzatoare fiecarui id din setul de date

def get_labels(data):
    new_data = []
    
    for item in data:
        new_data.append(item[1])
        
    #Vectorul cu toate etichetele este convertit intr-un numpy array   
    return np.array(new_data)

In [5]:
#Functie care converteste datele vectorizate in DataFrame-uri si adauga ultimele caracteristicii
#Parametrii: row: vectorul cu toate propozitiile
#vect: Matricea rezultata de vectorizarea datelor
#names: Lista caracteristicilor extrase prin vectorizare
def final_features(row, vect, names):

    #Transformarea datelor vectorizatre intr-o structura DataFrame din libraira pandas
    df = pd.DataFrame(data=vect, columns = names)
    
    #Transformarea propozitiilor intr-o structura DataFrame din libraira pandas
    txt = pd.DataFrame(row, columns = ['data'])
    
    #Lista cu vocalele limbii engleze
    vowels=['a','e','i','o','u']
    
    #Lista cu semnele de punctuatie uzuale
    punc = ["!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?"]
    
    #Functie care calculeaza densitate vocalelelor in propozitii
    def vowels_density(txt):
        
        #Calculeaza numarul total de caractere diferite de " "
        chars = len(txt.replace(" ", ""))
        
        #Calculeaza numarul de vocale dintr-o propozitie
        nr_vowels = sum([1 for c in txt if c in vowels])
        
        return(nr_vowels / chars)
    
    #Functie care calculeaza densitate semnelor de punctuatie in propozitii
    def punc_density(txt):
        
        #Calculeaza numarul total de caractere diferite de " "
        chars = len(txt.replace(" ", ""))
        
        #Calculeaza numarul semnelor de punctuatie dintr-o propozitie
        nr_punc = sum([1 for c in txt if c in punc])
        
        return(nr_punc / chars)

    def num_unique_words(txt):
        
        #Se imparte propozitia in cuvinte
        words = txt.split()
        
        #Se aleg cuvintele unice
        unique = set(w for w in words)
        
        #Retruneaza lungimea multimii de cuvinte
        return len(unique)
    
    #Se introduc noile caracteristici in DataFrame
    df['punc_density'] = txt['data'].apply(lambda txt: punc_density(txt))
    df['vowels_density'] = txt['data'].apply(lambda txt: vowels_density(txt))
    df['num_unique_words'] = txt['data'].apply(lambda txt: num_unique_words(txt))
    
    return df

In [6]:
#Functie care determina cel mai bun hyper parametru pentru predictie

def best_param(results):
    best = (0, 0)
    
    for data in results:
        if data[1] > best[1]:
            best = data
            
    return best

In [7]:
#Functie care scrie un fisier . csv cu predictile obtinute
#Parametrii: test_data: Datele intiale intr-un numpay array
#predicted: lista predictilor

def write_prediction(test_data, predicted):
    
    #Headerul tabelului
    header = ['id', 'label']
    
    #Vector cu datele care vor fii scrise
    data = []
    
    #Parcurgerea datelor initiale
    for i in range(0, len(test_data)):
        #Extragerea id-ului pentru fiecare propozitie
        id = int(test_data[i][0])
        #Extragerea etichetei prezise
        label = int(predicted[i])
        #Adaugarea datelor in vector
        data.append([id, label])

    #Generarea fiserului .csv
    with open('submisie.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        #Scriere header-ului
        writer.writerow(header)
        
        #Scrierea datelor in fisier
        for item in data:
            writer.writerow(item)

In [11]:
#Citirea si prelucarea datelor de antrenare si validare

#Citirea datelor de antrenare
training_data = np.genfromtxt('data_aliens/train_samples.txt', dtype='str', comments='\\\\\\\\\\\\\\\\', delimiter='\t', invalid_raise = False)
training_labels = np.loadtxt('data_aliens/train_labels.txt', dtype='int')

#Citirea datelor de validare
validation_data = np.genfromtxt('data_aliens/validation_samples.txt', dtype='str', comments='\\\\\\\\\\\\\\\\', delimiter='\t', invalid_raise = False)
validation_labels = np.loadtxt('data_aliens/validation_labels.txt', dtype='int')

#Citirea datelor de test
test_data_init = np.genfromtxt('data_aliens/test_samples.txt', dtype='str', comments='\\\\\\\\\\\\\\\\', delimiter='\t', invalid_raise = False)

#Prelucrarea datelor de antrenare
#Extragerea propozitiilor
x_train = get_sentence(training_data)

#Extragerea etichetelor
y_train = get_labels(training_labels)

#Prelucrarea datelor de validare
#Extragerea propozitiilor
x_valid = get_sentence(validation_data)

#Extragerea etichetelor
y_valid = get_labels(validation_labels)

#Prelucrarea datelor de test
#Extragerea propozitiilor
x_test = get_sentence(test_data_init)

In [12]:
#Vectorizarea datelor

#Initializarea CountVectorizer-ului cu numar maxim de carcteristici = 3000
count_vect = CountVectorizer(lowercase = False, max_features=3000, strip_accents=None, min_df=1, stop_words=None)

#Generarea vocabularului si extragerea carcteristicilor din datele de antrenare
X_train_counts = count_vect.fit_transform(x_train)

#Extragerea denumirilor caracteristicilor
names = count_vect.get_feature_names_out()

#Extragerea carcteristicilor din datele de validare
X_valid_counts = count_vect.transform(x_valid)

#Transformarea datelor vectorizatre in numpay array
train_data = X_train_counts.toarray()
valid_data = X_valid_counts.toarray()

In [13]:
#Transformarea in DataFrame

#Data frame cu toate caracteristicile pentru datele de antrenare
train_df = final_features(x_train, train_data, names)

#Data frame cu toate caracteristicile pentru datele de validare
valid_df = final_features(x_valid, valid_data, names)

In [14]:
#Normalizarea datelor (daca este cazul)

scale_train, scale_validation = normalize_data(train_df, valid_df)

No scaling was performed. Raw data is returned.


In [16]:
#Antrenarea modelului cu diferiti hyper parametrii

#Parametrii SVM
param = [1, 3, 5, 7, 10, 15]

results = []

for c in param:
    svm_classifier = svm.SVC(C=c, kernel='linear')
    
    svm_classifier.fit(scale_train, y_train)
    
    predicted = svm_classifier.predict(scale_validation)
    
    print(confusion_matrix(y_valid, predicted, labels=[1, 2, 3]))
    
    results.append((c, accuracy_score(predicted, y_valid)))
    
print(results)

best = best_param(results)

[[1312  430  258]
 [ 529  793  178]
 [ 391  226  883]]
[[1313  422  265]
 [ 561  757  182]
 [ 395  222  883]]
[[1303  428  269]
 [ 567  758  175]
 [ 381  231  888]]
[[1299  428  273]
 [ 559  768  173]
 [ 388  224  888]]
[[1293  433  274]
 [ 569  758  173]
 [ 392  222  886]]
[[1285  439  276]
 [ 566  759  175]
 [ 392  222  886]]
[(1, 0.5976), (3, 0.5906), (5, 0.5898), (7, 0.591), (10, 0.5874), (15, 0.586)]


In [17]:
print(best)

(1, 0.5976)


In [18]:
#Extragerea caracteristicilor din datele de testare
X_test_counts = count_vect.transform(x_test)

#Transformarea datelor vectorizatre in numpay array
test_data = X_test_counts.toarray()

#Data frame cu toate caracteristicile pentru datele de testare
test_df = final_features(x_test, test_data, names)

#Normalizarea datelor (daca este cazul)
scale_test, scale_test = normalize_data(test_df, test_df)

No scaling was performed. Raw data is returned.


In [21]:
svm_classifier = svm.SVC(C=best[0], kernel='linear')
    
svm_classifier.fit(scale_train, y_train)

predicted = svm_classifier.predict(scale_validation)

print(accuracy_score(predicted, y_valid))

print(confusion_matrix(y_valid, predicted, labels=[1, 2, 3]))

0.5976
[[1312  430  258]
 [ 529  793  178]
 [ 391  226  883]]


In [23]:
#Generarea predictilor pentru datele de test

predicted = svm_classifier.predict(scale_test)

In [24]:
#Scrierea fisierului .csv cu predictile obtinute

write_prediction(test_data_init, predicted)
print(predicted.shape)

(5000,)
